In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import time
import numpy as np


from sklearn.metrics import roc_curve, roc_auc_score, classification_report, f1_score, confusion_matrix, ConfusionMatrixDisplay, make_scorer, precision_score, recall_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import metrics

In [ ]:
df = pd.read_csv("alzheimers_disease_data.csv")
df

In [ ]:
df.info()

In [ ]:
alzheimer = df.drop(columns="DoctorInCharge")
alzheimer = alzheimer.drop(columns='PatientID')
alzheimer

In [ ]:
sns.countplot(x='Diagnosis', data=alzheimer)

In [ ]:
var_numeriques = [col for col in alzheimer.columns if alzheimer[col].nunique() > 10] #Considerem que les variables amb més de 10 valors diferents com variables numeriques.
var_categoriques = alzheimer.columns.difference(var_numeriques).difference(['Diagnosis']).to_list()

if 'PatientID' in var_numeriques:
    var_numeriques.remove('PatientID')

for var in var_categoriques:
    plt.figure(figsize=(6,4))
    sns.countplot(x="Diagnosis", hue=var, data=alzheimer)
    plt.title(f"Count Plot of {var}")
    plt.show()

In [ ]:
# Boxplots for numerical features by Diagnosis
for var in var_numeriques:
    plt.figure(figsize=(8,5))
    sns.boxplot(x='Diagnosis', y=var, data=df)
    plt.title(f'{var} by Diagnosis')
    plt.xticks([0, 1], ['No Alzheimer\'s', 'Alzheimer\'s'])
    plt.show()


In [ ]:
sns.pairplot(df[var_numeriques + ['Diagnosis']], hue='Diagnosis')
plt.show()


In [ ]:
f,ax = plt.subplots(figsize=(25, 25))
sns.heatmap(alzheimer.corr(), annot=True, linewidths= .5, fmt='.3f', ax=ax)
plt.show()

In [ ]:
alzheimer_long = alzheimer[var_numeriques].melt(var_name='Variable', value_name='Value')

plt.figure(figsize=(30,15))
sns.boxplot(x='Variable', y='Value', data=alzheimer_long)
plt.xticks(rotation=45)
plt.title("Boxplots de totes les variables numèriques")
plt.tight_layout()
plt.show()

In [ ]:
for var in var_numeriques:
    Q1 = alzheimer[var].quantile(0.25)
    Q3 = alzheimer[var].quantile(0.75)
    IQR = Q3 - Q1


    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = alzheimer[(alzheimer[var] < lower_bound) | (alzheimer[var] > upper_bound)]
    outliers.info()

In [ ]:
col = 'CholesterolTriglycerides'

Q1 = alzheimer[col].quantile(0.25)
Q3 = alzheimer[col].quantile(0.75)
IQR = Q3 - Q1


lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(lower_bound)
print(upper_bound)

outliers = alzheimer[(alzheimer[col] < lower_bound) | (alzheimer[col] > upper_bound)]
outliers
